In [1]:
from __future__ import annotations

%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path


from wufam.data.prepare_data import read_kf_data
from wufam.dataset import Dataset

PATH = Path("../data/kf_data")
START = "1970-01-01"
END = "2024-12-31"
DATASET = Dataset.BM_6_M
FACTORS_DATASET = Dataset.FACTORS_M
WEIGHTING = "value_weighted"
FACTOR_ANNUALIZE = 12

## 1.1

In [3]:
portfolios_total_r, portfolios_xs_r, factors_df, rf = read_kf_data(
    portfolios_filename=PATH / DATASET,
    factors_filename=PATH / FACTORS_DATASET,
    start_date=START,
    end_date=END,
    weighting=WEIGHTING,
)

In [4]:
assert (
    portfolios_total_r.shape[0]
    == portfolios_xs_r.shape[0]
    == factors_df.shape[0]
    == rf.shape[0]
)

## 1.4

In [5]:
from wufam.ap.uncond_factor_model import UncondFactorModel

capm = UncondFactorModel()
capm.fit(
    test_assets_xs_r=portfolios_xs_r,
    factors_df=factors_df[["Mkt-RF"]],
)

In [6]:
capm.grs_stat, capm.p_value

(np.float64(6.629569676484499), np.float64(8.082971012380241e-07))

Try monthly

In [7]:
ff_model = UncondFactorModel()
ff_model.fit(
    test_assets_xs_r=portfolios_xs_r,
    factors_df=factors_df,
)

In [8]:
ff_model.grs_stat, ff_model.p_value

(np.float64(5.26895705119617), np.float64(2.5726415393510304e-05))

## 1.5

In [9]:
split_date = portfolios_xs_r.index[len(portfolios_xs_r) // 2]

In [10]:
capm.fit(
    test_assets_xs_r=portfolios_xs_r.loc[:split_date],
    factors_df=factors_df[["Mkt-RF"]].loc[:split_date],
)

In [11]:
capm.rmse_score(
    test_assets_xs_r=portfolios_xs_r.loc[split_date:],
    factors=factors_df[["Mkt-RF"]].loc[split_date:],
)

np.float64(0.0014909569646982166)

In [12]:
capm.r2_score(
    test_assets_xs_r=portfolios_xs_r.loc[split_date:],
    factors=factors_df[["Mkt-RF"]].loc[split_date:],
)

np.float64(-1.1595217020999948)

In [13]:
capm.r2_gls_score(
    test_assets_xs_r=portfolios_xs_r.loc[split_date:],
    factors=factors_df[["Mkt-RF"]].loc[split_date:],
)

np.float64(-0.3658270839465645)

In [14]:
ff_model.fit(
    test_assets_xs_r=portfolios_xs_r.loc[:split_date],
    factors_df=factors_df.loc[:split_date],
)

In [15]:
ff_model.rmse_score(
    test_assets_xs_r=portfolios_xs_r.loc[split_date:],
    factors=factors_df.loc[split_date:],
)

np.float64(0.0010301585243249142)

In [16]:
ff_model.r2_score(
    test_assets_xs_r=portfolios_xs_r.loc[split_date:],
    factors=factors_df.loc[split_date:],
)

np.float64(-0.030944955328074064)

In [17]:
ff_model.r2_gls_score(
    test_assets_xs_r=portfolios_xs_r.loc[split_date:],
    factors=factors_df.loc[split_date:],
)

np.float64(-0.24602739512673666)